In [124]:
import os, shutil, re, datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from scipy.signal import resample
import librosa
import librosa.display
from helpers import *
import pylab

%matplotlib inline

In [125]:

FNAME = "0104_1522"
CH1_MAX = 270
CH2_MAX = 270
CH3_MAX = 220
CH4_MAX = 500
SAMPLE_RATE = 400
WINDOW = 2 # seconds of data per label
NB_CHANNELS = 2

SAVED_DATA= "/Users/kyy/openbci_stream/osc_data/"

VALID_LABELS = ['zero', 'one', 'two', 'three', 'four', 
            'five', 'six', 'seven', 'eight', 'nine',
            'left', 'right', 'stop', 'go', 'up', 
            'down', 'oneFinger', 'twoFinger', 'fourFinger']

CSV_ROOT = os.path.join(SAVED_DATA, FNAME) 
AUD_ROOT = os.path.join(os.getcwd(), "audio", FNAME)

print('-' * 42)
print("CSV Directory:   {}".format(CSV_ROOT))
print("AUDIO Directory: {}".format(AUD_ROOT))
print('-' * 42)

------------------------------------------
CSV Directory:   /Users/kyy/openbci_stream/osc_data/0104_1522
AUDIO Directory: /Users/kyy/cerebro_train/audio/0104_1522
------------------------------------------


In [126]:
def preprocess(samples, sample_rate, multiplier=1):
    sr = sample_rate * multiplier
    padded = np.zeros(sr)
    samples = samples[:sr]
    padded[:samples.shape[0]] = samples
    return padded

def make_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [127]:
def process_labels(input_dir, output_path):
    for label in [d for d in os.listdir(input_dir) if d in VALID_LABELS]:
        print("  Label Processing: {}".format(label))
        label_path = os.path.join(input_dir, label)
        o_label_path = os.path.join(output_dir, label)
        make_dir(o_label_path)
        for channel in range(1, NB_CHANNELS + 1):
            ch_dir = os.path.join(o_label_path, "ch{}".format(channel))
            make_dir(ch_dir)
        process_files(label_path, o_label_path)

In [128]:
def process_files(label_path, o_label_path):
    for file in [f for f in os.listdir(label_path) if f.endswith(".txt")]:
        csv_path = os.path.join(label_path, file)
        df = pd.read_csv(csv_path, ',')
        for channel in range(1, NB_CHANNELS + 1):
            ch_name = 'ch{}'.format(channel)
            ch_df = df[ch_name] 
            o_file_path = os.path.join(o_label_path, ch_name, file) 
            csv2wav(ch_name, ch_df, o_file_path)
            
            
#         wavpath = os.path.join(channel_path, file)
#         sample_rate, samples = wavfile.read(wavpath)
#         if MOVE_WINDOW == True:
#             items = make_windowed_spectrograms(samples, sample_rate, o_channel_path, items)
#         else:
#             items = make_spectrograms(samples, sample_rate, o_channel_path, items)

In [129]:
def csv2wav(ch_name, ch_data, o_file_path):
    
    data = np.array(ch_data, dtype='float64')
#     if ch_name == "ch1":
#         data /= CH1_MAX
#     if ch_name == "ch2":
#         data /= CH2_MAX
#     if ch_name == "ch3":
#         data /= CH3_MAX
#     if ch_name == "ch4":
#         data /= CH4_MAX
    print(np.max(np.abs(data)))
    data = data / np.max(np.abs(data))
    wav_path = o_file_path[:-4] + ".wav"
    wavfile.write(wav_path, 400, data)
    
#     data = signal.resample(data, (8000 // SAMPLE_RATE) * SAMPLE_RATE * WINDOW)
#     make_spectrograms(data, 8000, o_file_path[:-4] + ".png")

In [130]:
def make_spectrograms(samples, sample_rate, o_file_path): 
    changed = preprocess(samples, sample_rate, 1)
    S = librosa.feature.melspectrogram(changed, sr=sample_rate, n_mels=128, fmax=512)
    log_S = librosa.power_to_db(S, ref=np.max)
    fig = plt.figure(figsize=(1.28, 1.28), dpi=100, frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.axis('off')
    librosa.display.specshow(log_S)                          
    plt.savefig(o_file_path)
    plt.close()

In [131]:
if not os.path.isdir(AUD_ROOT):
    os.mkdir(AUD_ROOT)

dir_pairs = {
    CSV_ROOT : AUD_ROOT}

In [132]:
for input_dir in dir_pairs:
    output_dir = dir_pairs[input_dir]
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    timer(process_labels, input_dir, output_dir)

Start: 2019-01-04 08:36:55.682561
  Label Processing: oneFinger
734.38
734.38
1085.94
1085.94
937.5
937.5
718.75
718.75
898.44
898.44
1085.94
1085.94
914.06
914.06
921.88
921.88
1015.62
1015.62
656.25
656.25
656.25
656.25
992.19
992.19
710.94
710.94
710.94
710.94
1148.44
1148.44
1226.56
1226.56
1203.12
1203.12
742.19
742.19
703.12
703.12
1289.06
1289.06
656.25
656.25
1171.88
1171.88
1187.5
1187.5
687.5
687.5
734.38
734.38
1093.75
1093.75
703.12
703.12
1070.31
1070.31
750.0
750.0
1054.69
1054.69
726.56
726.56
703.12
703.12
1046.88
1046.88
703.12
703.12
859.38
859.38
851.56
851.56
695.31
695.31
828.12
828.12
640.62
640.62
640.62
640.62
914.06
914.06
757.81
757.81
695.31
695.31
695.31
695.31
703.12
703.12
703.12
703.12
796.88
796.88
1031.25
1031.25
898.44
898.44
812.5
812.5
734.38
734.38
  Label Processing: twoFinger
773.44
773.44
1101.56
1101.56
960.94
960.94
789.06
789.06
1046.88
1046.88
914.06
914.06
1046.88
1046.88
1101.56
1101.56
1843.75
1843.75
695.31
695.31
703.12
703.12
1132.81
11